In [ ]:
from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage

changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://eservice.nhi.gov.tw/nhiweb1/system/mLoginCa.aspx'

with open('Email.txt') as f:
    mails = f.read().split('\n')
while "" in mails:
    mails.remove("")
mails = set(mails)


def mailTo(title, mailAdds, message, whoSend='AutoDownloadList'):
    msg = MIMEMultipart()
    sender = whoSend
    subject = title
    body = message
    msg['From'] = sender
    msg['To'] = ','.join(mailAdds)
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    text = msg.as_string()
    # print text
    # Send the message via our SMTP server
    s = smtplib.SMTP('192.168.2.1', 25)
    s.sendmail(sender, mailAdds, text)
    s.quit()


def downloadAssureList():
    def renamefile(path):
        for pathTuple in os.walk(path):
            for file_ in pathTuple[2]:
                try:
                    if 'csv' in file_ and 'crdownload' not in file_:
                        name = []
                        uno = ""
                        savepath = ""
                        with open(os.path.join(pathTuple[0], file_), "r") as f:
                            datas = f.readlines()[1].split(",")
                            uno = mapAlias[datas[1]]
                            date = datas[0].replace("/", "")
                            style = "健保"
                            savepath = path + "\\健保"
                            if savepath and not os.path.exists(savepath):
                                os.makedirs(savepath)
                        try:
                            # print(file_)
                            if uno:
                                os.rename(os.path.join(pathTuple[0], file_),
                                          os.path.join(savepath, uno + style + "_" + date + ".csv"))
                        except Exception as er:
                            print(er)
                            print(os.path.join(path, file_) + "\t已有變更檔名的檔案")
                            os.remove(os.path.join(pathTuple[0], file_))
                except Exception as er:
                    report.append("更名失敗>>{},錯誤:{},出錯位置:{}".format(os.path.join(pathTuple[0], file_), er,
                                                                  sys.exc_info()[2].tb_lineno))

    report = []
    b = time.time()
    errorMsg = ""
    num = ""
    pinNum = e3.get()  # pin號碼
    chrome_options = Options()
    # crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
    # chrome_options.add_extension('extension_0_0_1_3.crx')

    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": e2.get(),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })

    # webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
    # 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon

    try:
        driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
        driver.get(web)

        dataDir = e1.get()
        inputPW = driver.find_element_by_id("USR_PASSWORD1")
        inputPW.clear()
        inputPW.send_keys(pinNum)
        driver.find_element_by_name("btnSubmit").click()

        alertText = ""
        try:
            x = driver.switch_to_alert()
            alertText = x.text
        except:
            pass
        if "其他錯誤" in alertText or "錯誤" in alertText:
            errorMsg = "密碼錯誤"
            print("密碼錯誤-登入失敗")
        else:
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))

        time.sleep(1)
        pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
        for excelFilePath in pathList:
            data = xlrd.open_workbook(excelFilePath)
            table = data.sheet_by_index(0)
            for i in range(10):
                try:
                    for text in table.col_values(i)[:7]:
                        #                 print(text+"---")
                        try:
                            if '店別' == text:
                                #                         print('xxx')
                                mapAliass = table.col_values(i)
                                # aliasNumberList = [str(int(no)) for no in mapAliass if
                                #                    no != "" and (type(no) == float or type(no) == int)]
                                # #                                 print(aliasNumberList)
                            if '健保投保單位代號' == text:
                                mapUno = table.col_values(i)
                                # print(text)
                                # unoNumberList = [str(int(no)) for no in mapUno if
                                #                  no != "" and type(no) == float]
                                # #                                 print(unoNumberList)
                                break
                            else:
                                continue
                        except:
                            pass
                except:
                    pass

        mapAlias = {str(int(alias)): str(int(uno)) for uno, alias in zip(mapAliass, mapUno) if
                    uno != "" and (type(uno) == float or type(uno) == int)
                    and alias != "" and type(alias) == float}

        print(mapAlias)

        n = 0
        print("共有{}筆證號要下載".format(len(mapAlias)))
        #前半部同樣的程序要跑兩次 第一次n=0跑申請 第二次n=1跑下載
        while n < 2:
            roundTime = 0
            for num in mapAlias:
                try:
                    try:
                        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'USR_PASSWORD1')))
                        time.sleep(0.2)
                        inputPW = driver.find_element_by_id("USR_PASSWORD1")
                        try:
                            inputPW.clear()
                        except:
                            pass
                        inputPW.send_keys(pinNum)
                        driver.find_element_by_name("btnSubmit").click()
                        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))
                        time.sleep(0.4)
                    except:
                        pass
                    driver.find_element_by_id("UNIT_ID").click()
                    time.sleep(0.2)
                    driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
                    driver.find_element_by_name("btnSubmit").click()
                    alertText = ""
                    try:
                        x = driver.switch_to_alert()
                        alertText = x.text
                        x.accept()
                    except:
                        pass
                    if "系統效期" in alertText or "單位使用本系統效期" in alertText:
                        if n == 0:
                            report.append("健保號碼:{}_沒權限無法登入".format(num))
                        continue

                    time.sleep(0.4)
                    nnnn = 0
                    tryNum = 0
                    while tryNum == 0:
                        try:
                            driver.switch_to_frame(0)
                            tryNum = 1
                        except:
                            nnnn += 1
                            if nnnn % 20 == 0:
                                tryNum = 1
                            time.sleep(0.3)
                    try:
                        if driver.find_element_by_id("MenuMaker1_TreeView1t30").text == "":
                            viewNum = 31
                        else:
                            viewNum = 30
                    except:
                        if num == "134862801":
                            viewNum = 29
                        elif num == "144619018":
                            viewNum = 14
                    if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
                        if num == "134862801":
                            viewNum = 29
                        elif num == "144619018":
                            viewNum = 14

                    driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).click()

                    #第一次跑時需要申請  n=0
                    if n == 0:
                        driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 1)).click()
                        try:
                            driver.find_element_by_id("ddl_ReportKind").click()
                        except:
                            pass
                        time.sleep(1)
                        driver.switch_to_frame(0)
                        driver.find_element_by_xpath('//option[@value="02"]').click()
                        time.sleep(0.3)
                        driver.find_element_by_id("ToolBar1_btnAdd").click()
                        time.sleep(1)
                        x = driver.switch_to_alert()
                        time.sleep(0.4)
                        try:
                            x.accept()
                        except:
                            pass
                        print("{}申請完畢".format(num))

                    # 第二次跑時需要下載  n=1
                    else:
                        roundTime += 1
                        if roundTime % 30 == 0:
                            renamefile(e2.get())
                        WebDriverWait(driver, 8).until(
                            EC.element_to_be_clickable((By.ID, "MenuMaker1_TreeView1t{}".format(viewNum + 2))))
                        time.sleep(0.3)
                        driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 2)).click()
                        time.sleep(1)
                        driver.switch_to_frame(0)
                        try:
                            driver.find_element_by_id("DropDownList1").click()
                        except:
                            pass

                        nnnn = 0
                        tryNum = 0
                        while tryNum == 0:
                            try:
                                driver.find_element_by_xpath('//option[@value="02"]').click()
                                tryNum = 1
                            except:
                                nnnn += 1
                                if nnnn % 20 == 0:
                                    tryNum = 1
                                time.sleep(0.3)

                        time.sleep(0.4)
                        driver.find_element_by_id("ToolBar1_btnQuery").click()
                        time.sleep(0.4)
                        try:
                            WebDriverWait(driver, 8).until(
                                EC.element_to_be_clickable((By.NAME, 'DataGrid1$ctl02$ctl00')))
                            time.sleep(0.4)
                            driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
                        except:
                            WebDriverWait(driver, 8).until(
                                EC.element_to_be_clickable((By.NAME, 'DataGrid1$ctl03$ctl00')))
                            time.sleep(0.4)
                            driver.find_element_by_name("DataGrid1$ctl03$ctl00").click()
                        time.sleep(0.3)
                        print("{}下載完畢".format(num))
                    driver.switch_to_default_content()
                    time.sleep(0.1)
                    driver.find_element_by_class_name("Logout").click()
                    time.sleep(0.5)
                except Exception as er:
                    print('健保清冊下載失敗-證號:{}'.format(num))
                    report.append("健保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(num, er, sys.exc_info()[2].tb_lineno))
                    try:
                        x = driver.switch_to_alert()
                        x.accept()
                    except:
                        pass
                    try:
                        driver.switch_to_default_content()
                    except:
                        pass
                    try:
                        driver.find_element_by_class_name("Logout").click()
                    except:
                        pass
                    try:
                        inputPW = driver.find_element_by_id("USR_PASSWORD1")
                        inputPW.clear()
                        inputPW.send_keys(pinNum)
                        driver.find_element_by_name("btnSubmit").click()
                    except:
                        pass

                    time.sleep(0.5)
            if n == 0:
                print("已申請完畢，開始等待30秒")
                for i in range(30):
                    time.sleep(1)
                    print("剩餘{}秒".format(30 - i))
            n += 1

        renamefile(e2.get())

        e = time.time()
        report.append("耗時{}秒".format(round(e - b)))
        report.append("共處理{}筆".format(len(mapAlias)))
        mailTo('ReportDownList', mails, web + "\n" + "\n".join(report))
        driver.quit()

    except Exception as er:
        if errorMsg == "":
            errorMsg = str(er) + "\n證號:{}".format(
                num) + '\nPlease verify website connection \n if no problem in website find andy.yuan@wowprime.com \n 出錯位置:' + str(
                sys.exc_info()[2].tb_lineno)

        print('出錯了RRRRRRRRRRRR' + str(er) + '\n')

        mailTo('ReportError', mails, errorMsg)


def selectPath():
    path_ = askdirectory()
    path.set(path_)


def selectPath2():
    path2_ = askdirectory()
    path2.set(path2_)


with open('defaultIO.txt') as f:
    IO = f.read()
defaultLoadDir = IO.split("\n")[0].split("=")[1].replace("\\", "/")
defaultSaveDir = IO.split("\n")[1].split("=")[1].replace("\\", "/")
root = Tk()
path = StringVar()
Label(root, text="讀取資料夾:").grid(row=0, column=0)
e1 = Entry(root, textvariable=path)
e1.insert(0, defaultLoadDir)
e1.grid(row=0, column=1)
Button(root, text="讀取資料夾", command=selectPath).grid(row=0, column=2)

path2 = StringVar()
Label(root, text="存放資料夾:").grid(row=1, column=0)
e2 = Entry(root, textvariable=path2)
e2.insert(0, defaultSaveDir)
e2.grid(row=1, column=1)

Button(root, text="存放資料夾", command=selectPath2).grid(row=1, column=2)

Label(root, text="自然人憑證PIN碼").grid(row=2, column=0)
e3 = Entry(root)
e3.grid(row=2, column=1)
# e1.insert(10,"Miller") #輸入預設值
Button(root, text='下載健保清冊開始', command=downloadAssureList).grid(row=3, column=1, pady=4)
root.mainloop()


In [5]:

from tkinter import Tk, StringVar, Label, Entry, Button
from tkinter.filedialog import askdirectory
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait  # 直到找到 不斷等待
from selenium.webdriver.support import expected_conditions as EC  # 判斷某物件存在，直到時間結束
import os
import sys
import time
import xl2dict
import smtplib
import xlrd
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.message import EmailMessage
changDict = {"4": "加保", "2": "退保", "3": "調保"}
web = 'https://eservice.nhi.gov.tw/nhiweb1/system/mLoginCa.aspx'

# with open('Email.txt') as f:
#     mails = f.read().split('\n')
# while "" in mails:
#     mails.remove("")
# mails = set(mails)


In [24]:
import time




report = []
b = time.time()
errorMsg = ""
num = ""
pinNum = 760205  # pin號碼
chrome_options = Options()
# crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
# chrome_options.add_extension('extension_0_0_1_3.crx')

chrome_options.add_experimental_option("prefs", {
    "download.default_directory": r"\\192.168.1.2\公用區\1819\aaa",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

In [25]:
driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
driver.get(web)

In [26]:
dataDir =r"\\192.168.1.2\公用區\1819\保險證號"
inputPW = driver.find_element_by_id("USR_PASSWORD1")
inputPW.clear()
inputPW.send_keys(pinNum)
driver.find_element_by_name("btnSubmit").click()

In [27]:
alertText = ""
try:
    x = driver.switch_to_alert()
    alertText = x.text
except:
    pass
if "其他錯誤" in alertText or "錯誤" in alertText:
    errorMsg = "密碼錯誤"
    print("密碼錯誤-登入失敗")
else:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))

time.sleep(1)
pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]

In [29]:
pathList

['\\\\192.168.1.2\\公用區\\1819\\保險證號\\各單位勞.健.勞退證號-6月.xlsx']

In [30]:
excelFilePath=pathList[0]

In [31]:
data = xlrd.open_workbook(excelFilePath)
table = data.sheet_by_index(0)
for i in range(10):
    try:
        for text in table.col_values(i)[:7]:
            #                 print(text+"---")
            try:
                if '店別' == text:
                    #                         print('xxx')
                    mapAliass = table.col_values(i)
                    # aliasNumberList = [str(int(no)) for no in mapAliass if
                    #                    no != "" and (type(no) == float or type(no) == int)]
                    # #                                 print(aliasNumberList)
                if '健保投保單位代號' == text:
                    mapUno = table.col_values(i)
                    # print(text)
                    # unoNumberList = [str(int(no)) for no in mapUno if
                    #                  no != "" and type(no) == float]
                    # #                                 print(unoNumberList)
                    break
                else:
                    continue
            except:
                pass
    except:
        pass

mapAlias = {str(int(alias)): str(int(uno)) for uno, alias in zip(mapAliass, mapUno) if
            uno != "" and (type(uno) == float or type(uno) == int)
            and alias != "" and type(alias) == float}

print(mapAlias)

n = 0
print("共有{}筆證號要下載".format(len(mapAlias)))

{'122776970': '10100', '131209820': '10202', '131780848': '10203', '142199942': '10204'}
共有4筆證號要下載


In [35]:
num='122776970'

In [37]:
driver.find_element_by_id("UNIT_ID").click()
time.sleep(0.2)
driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
driver.find_element_by_name("btnSubmit").click()
alertText = ""

In [39]:
driver.switch_to_frame(0)

In [41]:
viewNum = 31

In [43]:
if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
    if num == "134862801":
        viewNum = 29
    elif num == "144619018":
        viewNum = 14

In [49]:
driver.find_element_by_link_text("各類明細表申請").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"各類明細表申請"}
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.35.528161 (5b82f2d2aae0ca24b877009200ced9065a772e73),platform=Windows NT 6.1.7601 SP1 x86_64)


In [48]:
driver.switch_to_default_content()

In [ ]:
while n < 2:
    roundTime = 0
    for num in mapAlias:
        try:
            try:
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'USR_PASSWORD1')))
                time.sleep(0.2)
                inputPW = driver.find_element_by_id("USR_PASSWORD1")
                try:
                    inputPW.clear()
                except:
                    pass
                inputPW.send_keys(pinNum)
                driver.find_element_by_name("btnSubmit").click()
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))
                time.sleep(0.4)
            except:
                pass
            driver.find_element_by_id("UNIT_ID").click()
            time.sleep(0.2)
            driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
            driver.find_element_by_name("btnSubmit").click()
            alertText = ""
            try:
                x = driver.switch_to_alert()
                alertText = x.text
                x.accept()
            except:
                pass
            if "系統效期" in alertText or "單位使用本系統效期" in alertText:
                if n == 0:
                    report.append("健保號碼:{}_沒權限無法登入".format(num))
                continue

            time.sleep(0.4)
            nnnn = 0
            tryNum = 0
            while tryNum == 0:
                try:
                    driver.switch_to_frame(0)
                    tryNum = 1
                except:
                    nnnn += 1
                    if nnnn % 20 == 0:
                        tryNum = 1
                    time.sleep(0.3)
            try:
                if driver.find_element_by_id("MenuMaker1_TreeView1t30").text == "":
                    viewNum = 31
                else:
                    viewNum = 30
            except:
                if num == "134862801":
                    viewNum = 29
                elif num == "144619018":
                    viewNum = 14
            if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
                if num == "134862801":
                    viewNum = 29
                elif num == "144619018":
                    viewNum = 14

            driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).click()

            #第一次跑時需要申請  n=0
            if n == 0:
                driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 1)).click()
                try:
                    driver.find_element_by_id("ddl_ReportKind").click()
                except:
                    pass
                time.sleep(1)
                driver.switch_to_frame(0)
                driver.find_element_by_xpath('//option[@value="02"]').click()
                time.sleep(0.3)
                driver.find_element_by_id("ToolBar1_btnAdd").click()
                time.sleep(1)
                x = driver.switch_to_alert()
                time.sleep(0.4)
                try:
                    x.accept()
                except:
                    pass
                print("{}申請完畢".format(num))

            # 第二次跑時需要下載  n=1
            else:
                roundTime += 1
                if roundTime % 30 == 0:
                    renamefile(e2.get())
                WebDriverWait(driver, 8).until(
                    EC.element_to_be_clickable((By.ID, "MenuMaker1_TreeView1t{}".format(viewNum + 2))))
                time.sleep(0.3)
                driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 2)).click()
                time.sleep(1)
                driver.switch_to_frame(0)
                try:
                    driver.find_element_by_id("DropDownList1").click()
                except:
                    pass

                nnnn = 0
                tryNum = 0
                while tryNum == 0:
                    try:
                        driver.find_element_by_xpath('//option[@value="02"]').click()
                        tryNum = 1
                    except:
                        nnnn += 1
                        if nnnn % 20 == 0:
                            tryNum = 1
                        time.sleep(0.3)

                time.sleep(0.4)
                driver.find_element_by_id("ToolBar1_btnQuery").click()
                time.sleep(0.4)
                try:
                    WebDriverWait(driver, 8).until(
                        EC.element_to_be_clickable((By.NAME, 'DataGrid1$ctl02$ctl00')))
                    time.sleep(0.4)
                    driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
                except:
                    WebDriverWait(driver, 8).until(
                        EC.element_to_be_clickable((By.NAME, 'DataGrid1$ctl03$ctl00')))
                    time.sleep(0.4)
                    driver.find_element_by_name("DataGrid1$ctl03$ctl00").click()
                time.sleep(0.3)
                print("{}下載完畢".format(num))
            driver.switch_to_default_content()
            time.sleep(0.1)
            driver.find_element_by_class_name("Logout").click()
            time.sleep(0.5)
        except Exception as er:
            print('健保清冊下載失敗-證號:{}'.format(num))
            report.append("健保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(num, er, sys.exc_info()[2].tb_lineno))
            try:
                x = driver.switch_to_alert()
                x.accept()
            except:
                pass
            try:
                driver.switch_to_default_content()
            except:
                pass
            try:
                driver.find_element_by_class_name("Logout").click()
            except:
                pass
            try:
                inputPW = driver.find_element_by_id("USR_PASSWORD1")
                inputPW.clear()
                inputPW.send_keys(pinNum)
                driver.find_element_by_name("btnSubmit").click()
            except:
                pass

            time.sleep(0.5)
    if n == 0:
        print("已申請完畢，開始等待30秒")
        for i in range(30):
            time.sleep(1)
            print("剩餘{}秒".format(30 - i))
    n += 1

In [2]:
report = []
b = time.time()
errorMsg = ""
num = ""
pinNum = 760205  # pin號碼
chrome_options = Options()
# crx檔要用http://crxextractor.com/ 將套件網址key入，即可產生套件檔
# chrome_options.add_extension('extension_0_0_1_3.crx')

chrome_options.add_experimental_option("prefs", {
    "download.default_directory": e2.get(),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# webdriver檔要在 https://sites.google.com/a/chromium.org/chromedriver/downloads下載
# 套件網址:https://chrome.google.com/webstore/detail/smartpki-%E5%A4%9A%E6%86%91%E8%AD%89%E5%AE%89%E6%8E%A7%E6%A8%A1%E7%B5%84%E6%93%B4%E5%85%85%E5%A5%97%E4%BB%B6/jhffgcfmcckgmioipfnmbannkpncfipo?utm_source=chrome-ntp-icon


driver = webdriver.Chrome(executable_path='chromedriver.exe', chrome_options=chrome_options)
driver.get(web)

dataDir = e1.get()
inputPW = driver.find_element_by_id("USR_PASSWORD1")
inputPW.clear()
inputPW.send_keys(pinNum)
driver.find_element_by_name("btnSubmit").click()

alertText = ""
try:
    x = driver.switch_to_alert()
    alertText = x.text
except:
    pass
if "其他錯誤" in alertText or "錯誤" in alertText:
    errorMsg = "密碼錯誤"
    print("密碼錯誤-登入失敗")
else:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))

time.sleep(1)
pathList = [i[0] + "\\" + j for i in os.walk(dataDir) for j in i[2] if 'xlsx' in j and "$" not in j]
for excelFilePath in pathList:
    data = xlrd.open_workbook(excelFilePath)
    table = data.sheet_by_index(0)
    for i in range(10):
        try:
            for text in table.col_values(i)[:7]:
                #                 print(text+"---")
                try:
                    if '店別' == text:
                        #                         print('xxx')
                        mapAliass = table.col_values(i)
                        # aliasNumberList = [str(int(no)) for no in mapAliass if
                        #                    no != "" and (type(no) == float or type(no) == int)]
                        # #                                 print(aliasNumberList)
                    if '健保投保單位代號' == text:
                        mapUno = table.col_values(i)
                        # print(text)
                        # unoNumberList = [str(int(no)) for no in mapUno if
                        #                  no != "" and type(no) == float]
                        # #                                 print(unoNumberList)
                        break
                    else:
                        continue
                except:
                    pass
        except:
            pass

mapAlias = {str(int(alias)): str(int(uno)) for uno, alias in zip(mapAliass, mapUno) if
            uno != "" and (type(uno) == float or type(uno) == int)
            and alias != "" and type(alias) == float}

print(mapAlias)

n = 0
print("共有{}筆證號要下載".format(len(mapAlias)))
#前半部同樣的程序要跑兩次 第一次n=0跑申請 第二次n=1跑下載
while n < 2:
    roundTime = 0
    for num in mapAlias:
        try:
            try:
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, 'USR_PASSWORD1')))
                time.sleep(0.2)
                inputPW = driver.find_element_by_id("USR_PASSWORD1")
                try:
                    inputPW.clear()
                except:
                    pass
                inputPW.send_keys(pinNum)
                driver.find_element_by_name("btnSubmit").click()
                WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, 'UNIT_ID')))
                time.sleep(0.4)
            except:
                pass
            driver.find_element_by_id("UNIT_ID").click()
            time.sleep(0.2)
            driver.find_element_by_xpath('//option[@value="' + num + '"]').click()
            driver.find_element_by_name("btnSubmit").click()
            alertText = ""
            try:
                x = driver.switch_to_alert()
                alertText = x.text
                x.accept()
            except:
                pass
            if "系統效期" in alertText or "單位使用本系統效期" in alertText:
                if n == 0:
                    report.append("健保號碼:{}_沒權限無法登入".format(num))
                continue

            time.sleep(0.4)
            nnnn = 0
            tryNum = 0
            while tryNum == 0:
                try:
                    driver.switch_to_frame(0)
                    tryNum = 1
                except:
                    nnnn += 1
                    if nnnn % 20 == 0:
                        tryNum = 1
                    time.sleep(0.3)
            try:
                if driver.find_element_by_id("MenuMaker1_TreeView1t30").text == "":
                    viewNum = 31
                else:
                    viewNum = 30
            except:
                if num == "134862801":
                    viewNum = 29
                elif num == "144619018":
                    viewNum = 14
            if '下載' not in driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).text:
                if num == "134862801":
                    viewNum = 29
                elif num == "144619018":
                    viewNum = 14

            driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum)).click()

            #第一次跑時需要申請  n=0
            if n == 0:
                driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 1)).click()
                try:
                    driver.find_element_by_id("ddl_ReportKind").click()
                except:
                    pass
                time.sleep(1)
                driver.switch_to_frame(0)
                driver.find_element_by_xpath('//option[@value="02"]').click()
                time.sleep(0.3)
                driver.find_element_by_id("ToolBar1_btnAdd").click()
                time.sleep(1)
                x = driver.switch_to_alert()
                time.sleep(0.4)
                try:
                    x.accept()
                except:
                    pass
                print("{}申請完畢".format(num))

            # 第二次跑時需要下載  n=1
            else:
                roundTime += 1
                if roundTime % 30 == 0:
                    renamefile(e2.get())
                WebDriverWait(driver, 8).until(
                    EC.element_to_be_clickable((By.ID, "MenuMaker1_TreeView1t{}".format(viewNum + 2))))
                time.sleep(0.3)
                driver.find_element_by_id("MenuMaker1_TreeView1t{}".format(viewNum + 2)).click()
                time.sleep(1)
                driver.switch_to_frame(0)
                try:
                    driver.find_element_by_id("DropDownList1").click()
                except:
                    pass

                nnnn = 0
                tryNum = 0
                while tryNum == 0:
                    try:
                        driver.find_element_by_xpath('//option[@value="02"]').click()
                        tryNum = 1
                    except:
                        nnnn += 1
                        if nnnn % 20 == 0:
                            tryNum = 1
                        time.sleep(0.3)

                time.sleep(0.4)
                driver.find_element_by_id("ToolBar1_btnQuery").click()
                time.sleep(0.4)
                try:
                    WebDriverWait(driver, 8).until(
                        EC.element_to_be_clickable((By.NAME, 'DataGrid1$ctl02$ctl00')))
                    time.sleep(0.4)
                    driver.find_element_by_name("DataGrid1$ctl02$ctl00").click()
                except:
                    WebDriverWait(driver, 8).until(
                        EC.element_to_be_clickable((By.NAME, 'DataGrid1$ctl03$ctl00')))
                    time.sleep(0.4)
                    driver.find_element_by_name("DataGrid1$ctl03$ctl00").click()
                time.sleep(0.3)
                print("{}下載完畢".format(num))
            driver.switch_to_default_content()
            time.sleep(0.1)
            driver.find_element_by_class_name("Logout").click()
            time.sleep(0.5)
        except Exception as er:
            print('健保清冊下載失敗-證號:{}'.format(num))
            report.append("健保清冊下載失敗-證號:{},錯誤訊息:{},錯誤位置:{}".format(num, er, sys.exc_info()[2].tb_lineno))
            try:
                x = driver.switch_to_alert()
                x.accept()
            except:
                pass
            try:
                driver.switch_to_default_content()
            except:
                pass
            try:
                driver.find_element_by_class_name("Logout").click()
            except:
                pass
            try:
                inputPW = driver.find_element_by_id("USR_PASSWORD1")
                inputPW.clear()
                inputPW.send_keys(pinNum)
                driver.find_element_by_name("btnSubmit").click()
            except:
                pass

            time.sleep(0.5)
    if n == 0:
        print("已申請完畢，開始等待30秒")
        for i in range(30):
            time.sleep(1)
            print("剩餘{}秒".format(30 - i))
    n += 1

renamefile(e2.get())

e = time.time()
report.append("耗時{}秒".format(round(e - b)))
report.append("共處理{}筆".format(len(mapAlias)))
mailTo('ReportDownList', mails, web + "\n" + "\n".join(report))
driver.quit()

NameError: name 'time' is not defined